#Libraries

In [1]:
# install required packages
!pip install crewai langchain openai google-search-results python-dotenv langchain_community

In [2]:
  pip install -U duckduckgo-search

In [3]:
pip install gradio

#Import & API Key

In [4]:
import os
from typing import List
from crewai import Agent, Task, Crew, Process
from langchain.tools import DuckDuckGoSearchRun
from datetime import datetime

# Setup API key
os.environ['OPENAI_API_KEY'] = 'sk-proj-####'


#Crew Agent, Task , Tools

In [5]:
# search tool
class SearchTool:
    def __init__(self):
        self.search_engine = DuckDuckGoSearchRun()
        self.name = "web_search"
        self.description = "Searches the web for information about companies and their AI investments"

    def func(self, query: str) -> str:
        return self.search_engine.run(query)

search_tool = SearchTool()

In [6]:
# Researcher Agent
researcher = Agent(
    role='Company Research Specialist',
    goal='Find comprehensive information about companies investments in AI',
    backstory="""You are an expert at analyzing company announcements,
    press releases, and financial reports. You have extensive experience in
    tracking AI investments and initiatives across major corporations.""",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

# Report Writer Agent
report_writer = Agent(
    role='Business Report Writer',
    goal='Create concise and insightful reports about company AI investments',
    backstory="""You are a skilled business report writer with expertise in
    distilling complex information into clear, actionable insights. You excel
    at analyzing ROI and business impacts of technology investments.""",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)


In [7]:
def create_company_research_crew(query: str) -> dict:
    # Research Task
    research_task = Task(
        description=f"""
        Research task: {query}

        1. Search for and analyze recent announcements, press releases, and public
           communications from S&P 500 companies regarding AI investments in 2024
        2. For each company found, collect:
           - Company name
           - Investment details
           - Announced use cases
           - Any mentioned ROI metrics
        3. Verify information from multiple sources when possible
        4. Organize findings in a structured format

        Your response should be a structured list of companies and their AI initiatives.
        """,
        expected_output="""A structured list of S&P 500 companies with their AI investments in 2024,
        including detailed information about their initiatives, investments, and ROI metrics.""",
        agent=researcher
    )

    # Report Writing Task
    report_task = Task(
        description=f"""
        Using the research findings, create a comprehensive report that includes:
        1. A summary of the overall AI investment landscape among S&P 500 companies
        2. For each company:
           - 100-word summary of their AI use case
           - ROI insights where available
           - Sources for the information
        3. Organize the report in a clear, professional format

        The report should be exhaustive but concise, focusing on verified information.
        """,
        expected_output="""A comprehensive report detailing AI investments of S&P 500 companies in 2024,
        with summaries of use cases, ROI insights, and sources for each company.""",
        agent=report_writer
    )

    # crew
    crew = Crew(
        agents=[researcher, report_writer],
        tasks=[research_task, report_task],
        verbose=True
    )

    # Execute crew's tasks
    result = crew.kickoff()

    return result

#User Query

In [88]:
query = input("Please provide your query 🚀 :")
result = create_company_research_crew(query)
print(result)

Please provide your query 🚀 :Generate a report that mentions all the S&P 500 companies that invested in AI in 2024 in any publicly available communication. Provide a 100 word summary on their use case and any insights on ROI. Please make sure that the list returned is exhaustive.


# Agent: Company Research Specialist
## Task: 
        Research task: Generate a report that mentions all the S&P 500 companies that invested in AI in 2024 in any publicly available communication. Provide a 100 word summary on their use case and any insights on ROI. Please make sure that the list returned is exhaustive.

        1. Search for and analyze recent announcements, press releases, and public
           communications from S&P 500 companies regarding AI investments in 2024
        2. For each company found, collect:
           - Company name
           - Investment details
           - Announced use cases
           - Any mentioned ROI metrics
        3. Verify information from multiple sources when possible
        4. Organize findings in a structured format

        Your response should be a structured list of companies and their AI initiatives.
        
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation err

#Gradio UI

In [8]:
import gradio as gr

class SearchTool:
    def __init__(self):
        self.search_engine = DuckDuckGoSearchRun()
        self.name = "web_search"
        self.description = "Searches the web for information about companies and their AI investments"

    def func(self, query: str) -> str:
        return self.search_engine.run(query)

search_tool = SearchTool()

researcher = Agent(
    role='Company Research Specialist',
    goal='Find comprehensive information about companies investments in AI',
    backstory="""You are an expert at analyzing company announcements,
    press releases, and financial reports. You have extensive experience in
    tracking AI investments and initiatives across major corporations.""",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

report_writer = Agent(
    role='Business Report Writer',
    goal='Create concise and insightful reports about company AI investments',
    backstory="""You are a skilled business report writer with expertise in
    distilling complex information into clear, actionable insights. You excel
    at analyzing ROI and business impacts of technology investments.""",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

def create_company_research_crew(query: str) -> dict:
    research_task = Task(
        description=f"""
        Research task: {query}

        1. Search for and analyze recent announcements, press releases, and public
           communications from S&P 500 companies regarding AI investments in 2024
        2. For each company found, collect:
           - Company name
           - Investment details
           - Announced use cases
           - Any mentioned ROI metrics
        3. Verify information from multiple sources when possible
        4. Organize findings in a structured format

        Your response should be a structured list of companies and their AI initiatives.
        """,
        expected_output="""A structured list of S&P 500 companies with their AI investments in 2024,
        including detailed information about their initiatives, investments, and ROI metrics.""",
        agent=researcher
    )

    report_task = Task(
        description=f"""
        Using the research findings, create a comprehensive report that includes:
        1. A summary of the overall AI investment landscape among S&P 500 companies
        2. For each company:
           - 100-word summary of their AI use case
           - ROI insights where available
           - Sources for the information
        3. Organize the report in a clear, professional format

        The report should be exhaustive but concise, focusing on verified information.
        """,
        expected_output="""A comprehensive report detailing AI investments of S&P 500 companies in 2024,
        with summaries of use cases, ROI insights, and sources for each company.""",
        agent=report_writer
    )

    crew = Crew(
        agents=[researcher, report_writer],
        tasks=[research_task, report_task],
        verbose=True
    )

    result = crew.kickoff()
    return result

def generate_report(query):
    result = create_company_research_crew(query)
    return result

iface = gr.Interface(
    fn=generate_report,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query about S&P 500 companies' AI investments..."),
    outputs=gr.Textbox(label="Research Report"),
    title="S&P 500 AI Investment Research Tool",
    description="Enter a query to research AI investments of S&P 500 companies in 2024."
)

iface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://746d581a94d652d5fa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
